In [1]:
from jupyter_plotly_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash
from dash.dependencies import Input, Output
from pymongo import MongoClient
import urllib.parse
from bson.json_util import dumps
import dash_leaflet as dl
import plotly.express as px
import dash_table as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

# Imports AnimalShelter class from module
from CRUDDatabase import AnimalShelter

username = "aacuser"
password = "f"
shelter = AnimalShelter()
# Authorises user
shelter._init_(username, password)
    
# Creates a DataFrame object that contains all documents in the animals collection
df = pd.DataFrame.from_records(shelter.read({}))

# Creates a JupyterDash object
app = JupyterDash('Project Two')

# Creates variable for logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



# Creates a layout in HTML that establishes how the dashboard will be displayed.
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Displays logo and header
    html.Div(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height': '10%', 'width': '10%'}),style={'textAlign': 'center'}),
    html.Div(html.Center(html.B(html.H1('Ian Pruter SNHU CS-340 Dashboard')))),
    
    # Displays radio buttons for each filter
    html.Hr(),
    html.Div(dcc.RadioItems(
                id='radio-items',
                options = [
                    {'label': 'Water Rescue', 'value': '1'},
                    {'label': 'Mountain Rescue', 'value': '2'},
                    {'label': 'Disaster Rescue', 'value': '3'},
                    {'label': 'Reset', 'value': '4'},

                    ],
                value = '',
                labelStyle={'display': 'inline-block'}
    )), 
    # Displays table
    dt.DataTable(
        # Displays columns depending on user interaction
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # Convers documents to dictionaries to display in the dashboared
        data=df.to_dict('records'),
        # Features for the interactive data table
        editable = False,
        filter_action ="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',

                    ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )
            ]
        )
]
)

# Changes background color when mouse hovers over it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Reads from the database according to radio buttons selected
@app.callback(Output('datatable-id','data'),
    [Input('radio-items', 'value')])

def update_dashboard(filter_type):
    # Water rescue filter
    if (int(filter_type) == 1):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    # Mountain rescue filter    
    elif (int(filter_type) == 2):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    # Disaster rescue filter  
    elif (int(filter_type) == 3):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))) 
     # Reset   
    elif (int(filter_type) == 4):
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))    
    return df.to_dict('records')

#Adds a pie chart that updates according to the filter
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    values = dff.groupby(['breed']).size()
    names = dff.breed.unique()
    
    return [
        dcc.Graph(            
            figure = px.pie(dff, values=values, names=names)
        )    
    ]

# Displays map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]
    
# Calls the dashboard
app